# 第5章 合并

## 五、问题与练习

In [1]:
import numpy as np
import pandas as pd

### 1. 问题
#### 【问题一】 请思考什么是append/assign/combine/update/concat/merge/join各自最适合使用的场景，并举出相应的例子。
- append 增加整条的数据，用于向df表中增量更新数据
- assign 为现有df添加新数据列，分析研究过程中为df表增加新的参数
- combine 用于两张表的数据融合。df1.combine(df2):先将两表扩充至相同size和索引结构(outer，结果表顺序由df1确定，非空数据位置由df2主导）。描述同一组信息的两张表的信息结合，通过某些标准进行数据筛选。
- update 在现有df1的框架下更新数据，以增加数据信息量为标准，只取df2与df1索引交集部分更新df1中内容。使用新的总表数据对感兴趣的数据df做更新。
- concat 表的横向/纵向拼接，以横纵索引为标准，不会进行数据覆盖。多个相同性质表的简单合并扩展。
- merge 将【两个】pandas对象横向合并，通过指定【列或索引】进行对应合并，sql表操作功能，由键关联的不同表间查询。
- join merge的index版本快捷操作。仅以index为键的表间查询。



#### 【问题二】 merge_ordered和merge_asof的作用是什么？和merge是什么关系？
- merge_ordered merge的有序拼接，顺序由left/right_by指定。可使用fill_method填补合并表的空值
- merge_asof 有序的left-join，匹配key最近的键而不必须是相等的键，可设置key的容错范围tolerance。



#### 【问题三】 请构造一个多级索引与多级索引合并的例子，尝试使用不同的合并函数。


In [197]:
#多级索引合并的例子
df = pd.read_csv('data/table.csv',index_col='ID')
df_using_mul = df.set_index(['Class','Address'])
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [277]:
'''
层次化索引的合并，
Data copy from https://blog.csdn.net/qq_38788128/article/details/80761621
'''
lefth=pd.DataFrame({'key1':['Oh','Oh','Oh','ne','ne'],'key2':[2000,2001,2002,2001,2002],'data':np.arange(5)})
print(lefth)
righth=pd.DataFrame(np.arange(12).reshape(6,2),index=[['ne','ne','Oh','Oh','Oh','Oh'],[2001,2000,2000,2000,2001,2002]],columns=['event1','event2'])
print(righth)
righth2=pd.DataFrame(np.arange(4,16).reshape(6,2),index=[['ne','ne','Oh','Oh','Oh','Ah'],[2001,2000,2000,2000,2001,2002]],columns=['event2','event4'])
print(righth2)

  key1  key2  data
0   Oh  2000     0
1   Oh  2001     1
2   Oh  2002     2
3   ne  2001     3
4   ne  2002     4
         event1  event2
ne 2001       0       1
   2000       2       3
Oh 2000       4       5
   2000       6       7
   2001       8       9
   2002      10      11
         event2  event4
ne 2001       4       5
   2000       6       7
Oh 2000       8       9
   2000      10      11
   2001      12      13
Ah 2002      14      15


In [278]:
#merge index+columns
pd.merge(righth,lefth,right_on=['key1','key2'],left_index=True)
#键的选取：左边用列，右边用index。键的规模和数据类型要相同。index转化为columns

,event1,event2,key1,key2,data
3,0,1,ne,2001,3
0,4,5,Oh,2000,0
0,6,7,Oh,2000,0
1,8,9,Oh,2001,1
2,10,11,Oh,2002,2


In [279]:
#merge index index
pd.merge(righth,righth2,left_index=True,right_index=True)
#左右都使用index，转化后还是index

event1  event2_x  event2_y  event4
Oh 2000       4         5         8       9
   2000       4         5        10      11
   2000       6         7         8       9
   2000       6         7        10      11
   2001       8         9        12      13
ne 2000       2         3         6       7
   2001       0         1         4       5

In [280]:
#join 有相同columns name时会报错
#righth.join(righth2)

In [281]:
#append index index
righth.append(righth2).head()

event1  event2  event4
ne 2001     0.0       1     NaN
   2000     2.0       3     NaN
Oh 2000     4.0       5     NaN
   2000     6.0       7     NaN
   2001     8.0       9     NaN

In [282]:
#append index columns
righth.append(lefth)

,event1,event2,key1,key2,data
"(ne, 2001)",0.0,1.0,NaN,NaN,NaN
"(ne, 2000)",2.0,3.0,NaN,NaN,NaN
"(Oh, 2000)",4.0,5.0,NaN,NaN,NaN
"(Oh, 2000)",6.0,7.0,NaN,NaN,NaN
"(Oh, 2001)",8.0,9.0,NaN,NaN,NaN
"(Oh, 2002)",10.0,11.0,NaN,NaN,NaN
0,NaN,NaN,Oh,2000.0,0.0
1,NaN,NaN,Oh,2001.0,1.0
2,NaN,NaN,Oh,2002.0,2.0
3,NaN,NaN,ne,2001.0,3.0


In [283]:
#combine 要求索引结构类似
righth.combine(righth2,lambda x,y:x if x.mean()>y.mean() else y)

event1  event2  event4
Ah 2002     NaN    14.0    15.0
Oh 2000     NaN     8.0     9.0
   2000     NaN    10.0    11.0
   2000     NaN     8.0     9.0
   2000     NaN    10.0    11.0
   2001     NaN    12.0    13.0
   2002     NaN     NaN     NaN
ne 2000     NaN     6.0     7.0
   2001     NaN     4.0     5.0

In [284]:
#righth.update(righth2) #报错，无法处理索引不唯一情况

In [285]:
#concat 需要相关方向索引唯一
pd.concat([righth2,righth],join='outer')

event2  event4  event1
ne 2001       4     5.0     NaN
   2000       6     7.0     NaN
Oh 2000       8     9.0     NaN
   2000      10    11.0     NaN
   2001      12    13.0     NaN
Ah 2002      14    15.0     NaN
ne 2001       1     NaN     0.0
   2000       3     NaN     2.0
Oh 2000       5     NaN     4.0
   2000       7     NaN     6.0
   2001       9     NaN     8.0
   2002      11     NaN    10.0

#### 【问题四】 上文提到了连接的笛卡尔积，那么当连接方式变化时（inner/outer/left/right），这种笛卡尔积规则会相应变化吗？请构造相应例子。
- 构建了如下例子，未发现笛卡尔积规则有变化。根据how的选择锁定key的元素，对重复索引做笛卡尔积处理，inner部分条数展开为a*b条，当使用outer或left/right导致a=0时，展开条数为b条。

In [286]:
left = pd.DataFrame({'A': [1, 2,3], 'B': [1, 2,3]})
right = pd.DataFrame({'A': [4, 5, 6, 7,9,9], 'B': [1, 2, 2, 2, 3, 3]})
pd.merge(left, right, on='B', how='right')

,A_x,B,A_y
0,1,1,4
1,2,2,5
2,2,2,6
3,2,2,7
4,3,3,9
5,3,3,9


### 2. 练习
#### 【练习一】有2张公司的员工信息表，每个公司共有16名员工，共有五个公司，请解决如下问题：

In [2]:
comp1 = pd.read_csv('data/Employee1.csv')
comp2 = pd.read_csv('data/Employee2.csv')
display(comp1.describe(),comp2.describe())

,Age,Height,Weight,Salary
count,40.000000,40.000000,40.000000,40.00000
mean,44.525000,174.725000,76.847500,16169.72500
std,7.765761,10.310531,14.742699,7157.87817
min,32.000000,155.000000,51.900000,3246.00000
25%,37.000000,167.000000,63.475000,11716.25000
50%,44.500000,174.000000,77.250000,17212.00000
75%,50.250000,183.250000,91.600000,21615.50000
max,58.000000,190.000000,98.900000,29223.00000


,Age,Height,Weight,Salary
count,37.000000,37.000000,37.000000,37.000000
mean,45.432432,171.702703,78.116216,18324.567568
std,9.290200,11.647281,15.417225,6753.977587
min,30.000000,155.000000,51.000000,4549.000000
25%,37.000000,163.000000,66.500000,13092.000000
50%,47.000000,170.000000,79.400000,17396.000000
75%,54.000000,183.000000,91.500000,23604.000000
max,60.000000,190.000000,99.100000,29963.000000


#### (a) 每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。
#### (b) 将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。
#### (c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。

In [3]:
#(a)
comp12 = pd.merge(comp1,comp2,on = ['Company','Name'],how='outer', indicator=True)
comp12[comp12._merge == 'both'].groupby('Company')['Name'].count()

Company
A    3
B    4
C    4
D    2
E    3
Name: Name, dtype: int64

In [32]:
#(b)只出现一次的员工列表
nametwice = (comp12[comp12._merge == 'both']['Name'])
comp1_ = comp1[~comp1['Name'].isin(nametwice)]
comp2_ = comp2[~comp2['Name'].isin(nametwice)]
comp12_ = pd.concat([comp1_,comp2_]).sort_values(by=['Name'])
comp12_.tail()

,Company,Name,Age,Height,Weight,Salary
30,E,e2,60,183,67.3,15515
31,E,e3,40,186,99.1,9976
33,E,e4,37,167,91.3,19171
32,E,e6,44,165,91.5,28818
34,E,e7,44,164,51.9,13035


In [92]:
'''(c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，
对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，
例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，
    且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，
    最后对于没有信息的员工，利用缺失值填充。
'''
comp12b = comp12_.copy()
stand_b = comp12b.groupby('Company').mean()#计算公司均值标准

bothname = comp12[comp12._merge == 'both']#重复名单，待处理
for C in ['A','B','C','D','E']:#清洗出现两次的
    result = bothname[bothname['Company']==C].copy()
    for part in ['Age','Height','Weight','Salary']:
        partx = part+'_x';party=part+'_y';mean  = stand_b.loc[C,part]
        result[part] = result.apply(lambda x: x[partx] if abs(x[partx]-mean)< abs(x[party]-mean) else x[party],axis=1)
    result.drop(columns=['Age_x', 'Height_x', 'Weight_x', 'Salary_x', \
                         'Age_y','Height_y', 'Weight_y','Salary_y', '_merge'],inplace=True)
    comp12b = pd.concat([comp12b,result])

comp12b#去重完成

,Company,Name,Age,Height,Weight,Salary
5,A,a10,56.0,164.0,82.2,24799.0
6,A,a12,36.0,177.0,96.9,10270.0
7,A,a13,58.0,190.0,75.9,21854.0
7,A,a14,56.0,190.0,62.6,10229.0
8,A,a15,35.0,162.0,97.2,4706.0
...,...,...,...,...,...,...
28,D,d5,50.0,173.0,85.7,14108.0
31,D,d10,50.0,158.0,70.3,21569.0
35,E,e8,45.0,171.0,68.1,20836.0
36,E,e10,50.0,186.0,76.0,17070.0


In [129]:
#用公司均值填补空值
company=list();staff=list()
for c in ['A','B','C','D','E']:
    for s in range(0,16):
        company.append(c)
        staff.append(c.lower()+str(s+1))
complete = pd.merge(comp12b,pd.DataFrame({'Company':company,'Name':staff}),how='outer')
for part in ['Age','Height','Weight','Salary']:
    complete[part] = complete.apply(lambda x: x[part] if  x[part]>0 else int(stand_b.loc[x['Company'],part]),axis=1)
complete.sort_values(by='Name')

,Company,Name,Age,Height,Weight,Salary
45,A,a1,47.0,188.0,91.2,25819.0
0,A,a10,56.0,164.0,82.2,24799.0
61,A,a11,46.0,174.0,81.0,12725.0
1,A,a12,36.0,177.0,96.9,10270.0
2,A,a13,58.0,190.0,75.9,21854.0
...,...,...,...,...,...,...
76,E,e5,46.0,170.0,77.0,19049.0
43,E,e6,44.0,165.0,91.5,28818.0
44,E,e7,44.0,164.0,51.9,13035.0
58,E,e8,45.0,171.0,68.1,20836.0


#### 【练习二】有2张课程的分数表（分数随机生成），但专业课（学科基础课、专业必修课、专业选修课）与其他课程混在一起，请解决如下问题：

In [133]:
course1 = pd.read_csv('data/Course1.csv')

In [134]:
course2 = pd.read_csv('data/Course2.csv')

#### (a) 将两张表分别拆分为专业课与非专业课（结果为四张表）。
#### (b) 将两张专业课的分数表和两张非专业课的分数表分别合并。
#### (c) 不使用(a)中的步骤，请直接读取两张表合并后拆分。
#### (d) 专业课程中有缺失值吗，如果有的话请在完成(3)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。

In [141]:
#(a)
majorlist = ['学科基础课','专业必修课','专业选修课']
course11 = course1[ course1['课程类别'].isin(majorlist)].copy()
course12 = course1[~course1['课程类别'].isin(majorlist)].copy()
course21 = course2[ course2['课程类别'].isin(majorlist)].copy()
course22 = course2[~course2['课程类别'].isin(majorlist)].copy()
display(course11.head(),course12.head(),course21.head(),course22.head())

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0


,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0


,课程名字,课程类别,学分,分数
25,学术英语听说（二）,英语类,2,92.0
26,学术英语阅读,英语类,2,72.0
27,学术英语写作,英语类,2,98.0
28,美国社会与文化,英语类,2,77.0
29,马克思主义基本原理概论,思政类,3,95.0


In [149]:
#(b)
pd.concat([course11,course21]).shape#专业课
pd.concat([course12,course22]).shape#非专业课

(21, 4)

In [171]:
#(c/d)
course = pd.concat([course1,course2])#合并表格
course['分数'] = course.groupby('课程类别').transform(lambda x:x.fillna(x.mean()))
#transform用法需要熟悉
course1 = course[course['课程类别'].isin(majorlist)]
course2 = course[~course['课程类别'].isin(majorlist)]